- обработала большую часть xml
- пока складываю в postgresql:
    - не факт, что оставлю, но проще потом убрать
    - удобно хранить - сущности со связями (пока просто несвязанные таблицы, основные связи будут по ИНН)
    - удобно делать пакетную вставку при обработке пофайлово
    
- больше всего времени заняло подключение через коннектор - потому что пустая OperationalError: (кажется, параметры идентификации - в конфигах сработало изменить md5 вместо scha...)

In [1]:
import os
import pickle
import requests

from lxml import etree

#### -  xml с реестром мсп (малых и средних предприятий)

https://www.nalog.gov.ru/opendata/7707329152-rsmp/

Структура документа:
- все объекты Документ, внутри которых есть ОргВклМСП

- итерируясь по этим объектам забираем:
    - Документ: 
        - атрибуты ССЧР (Сведения о среднесписочной численности работников), СведСоцПред (Сведения о том, что юридическое лицо / индивидуальный предприниматель является социальным предприятием, 1,2), ПризНовМСП (Признак сведений о вновь созданном юридическом лице / вновь зарегистрированном индивидуальном предпринимателе, 1,2), КатСубМСП (Категория субъекта МСП - ?, 1,2,3), ВидСубМСП (Вид субъекта МСП - ?, 1,2), ДатаВклМСП
        - объект ОргВклМСП атрибут ИННЮЛ - будет ключом
        - объект СвОКВЭД
            - вложенные объекты - если есть СвОКВЭДДоп - из первого атрибут ВерсОКВЭД (версия справочника),
            по остальным - список атрибутов КодОКВЭД
        - далее - если есть СвЛиценз (сведения о лицензиях) - счётчик, если несколько: считаем вложенные объекты НаимЛицВД (наименование лицензии)

        - СвПрод (Сведения о производимой субъектом МСП продукции) - если есть, мб несколько объектов - собираем в список атрибут КодПрод (Код вида продукции) (это справочник ОКПД2?), если есть хотя бы 1 атрибут ПрОтнПрод == 1 (признак отнесения продукциик инновационной, высокотехнологичной) - можно взять как бинарный признак

        - объект СвКонтр - мб несколько (Сведения о наличии у субъекта МСП в предшествующем календарном году контрактов, заключенных в соответствии с Федеральным законом от 5 апреля 2013 года №44-ФЗ) - для обеспечения государственных и муниципальных нужд: взять число контрактов

        - СвДог (Сведения о наличии у субъекта МСП в предшествующем календарном году договоров, заключенных в соответствии с Федеральным законом от 18 июля 2011 года №223-ФЗ) - О закупках товаров, работ, услуг: мб несколько, взять число

кажется, 1 - это да, 2 - нет

In [11]:
file = "../raw_data/reestr_ms/data-10032023-structure-10032022/VO_RRMSPSV_0000_9965_20230310_00a62ea7-b905-4787-aec9-3d44f97cdb1f.xml"
tree = etree.parse(file)
root = tree.getroot()
docs = root.findall("Документ[ОргВклМСП]")
doc = docs[1]

In [12]:
print(etree.tostring(doc, encoding = "unicode", pretty_print=True))

<Документ ИдДок="308515e2-59fe-4000-b750-8afdd4f6e095" ДатаСост="10.03.2023" ДатаВклМСП="10.09.2019" ВидСубМСП="1" КатСубМСП="1" ПризНовМСП="2" СведСоцПред="2" ССЧР="6">
  <ОргВклМСП НаимОрг="ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot;ТЕКЛАЙН&quot;" НаимОргСокр="ООО &quot;ТЕКЛАЙН&quot;" ИННЮЛ="4702020559" ОГРН="1194704019894"/>
  <СведМН КодРегион="47">
    <Регион Тип="ОБЛАСТЬ" Наим="ЛЕНИНГРАДСКАЯ"/>
    <Район Тип="РАЙОН" Наим="ВОЛХОВСКИЙ"/>
    <Город Тип="ГОРОД" Наим="ВОЛХОВ"/>
  </СведМН>
  <СвОКВЭД>
    <СвОКВЭДОсн КодОКВЭД="42.11" НаимОКВЭД="Строительство автомобильных дорог и автомагистралей" ВерсОКВЭД="2014"/>
    <СвОКВЭДДоп КодОКВЭД="77.32" НаимОКВЭД="Аренда и лизинг строительных машин и оборудования" ВерсОКВЭД="2014"/>
    <СвОКВЭДДоп КодОКВЭД="52.29" НаимОКВЭД="Деятельность вспомогательная прочая, связанная с перевозками" ВерсОКВЭД="2014"/>
    <СвОКВЭДДоп КодОКВЭД="46.73.6" НаимОКВЭД="Торговля оптовая прочими строительными материалами и изделиями" ВерсОКВЭД="2014"/>
 

In [2]:
# небольшая очистка
def clean(val):
    if isinstance(val, str):
        val = val.strip()
#         if val.isdigit():
#             return int(val)
    return val

- неожиданная проблема возникла после очистки инн - неуникальные инн (в итоге - всего 2): проще разрешить неуникальность и потом вычислить такие инн, чем проверять каждый инн до формирования пакета
- поэтому делаю временный primary key для таблицы, а после вставки - вычищаю повторы и изменяю ключ

In [3]:
# таблица reestr_msp
# Критерии среднего предприятия - не более 1500 человек
# формат даты распознаёт и приводит автоматически
reestr_sql = 'CREATE TABLE reestr_msp (\
temp_id bigserial PRIMARY KEY, \
inn int8, \
n_employees int2, \
is_soc_enterprise int2, \
newly_establ int2, \
subj_cat int2, \
subj_form int2, \
inclusion_date date, \
n_okovd_add int2, \
okovd_add_vers varchar (8), \
okovd_add_code text [], \
n_licenses int2, \
n_products int2, \
products_code text [], \
is_high_tech int2, \
n_contracts int2, \
n_agreements int2 \
);'

In [ ]:
# обёртка для транзакции (вместо commit())
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
with connect:
    with connect.cursor() as cur:
        cur.execute(reestr_sql)

connect.close()

In [4]:
# функция - для получения одного val
def get_val(doc):
    """
    Принимает один документ
    Возвращает необходимые поля, упакованые в кортеж
    """
    val = []
    # это ключ
    val.append(clean(doc.find('ОргВклМСП').attrib['ИННЮЛ']))
    # некоторых атрибутов может не быть
    try:
        val.append(clean(doc.attrib['ССЧР']))
    except KeyError:
        val.append(None)
    try:
        # является ли соц. предприятием
        val.append(1 if doc.attrib['СведСоцПред'] == '1' else 0)
    except KeyError:
        val.append(None)        
    try:
        # вновь создано?
        val.append(1 if doc.attrib['ПризНовМСП'] == '1' else 0 )
    except KeyError:
        val.append(None)
    try:    
        # категория организации
        val.append(clean(doc.attrib['КатСубМСП']))  # 1, 2, 3, не знаю, что - эти двое совпадают?
    except KeyError:
        val.append(None)    
    try:    
        # форма
        val.append(clean(doc.attrib['ВидСубМСП'])) # 1, 2, 3, не знаю, что
    except KeyError:
        val.append(None)
    try:
        # дата включения в реестр
        val.append(doc.attrib['ДатаВклМСП'])
    except KeyError:
        val.append(None)

    # а здесь - по видам деятельности, код основного - из mongo, а из дополнительных - если есть:
    # если ничего нет, ошибки не будет
    add_occupations = doc.findall('СвОКВЭД/СвОКВЭДДоп')
    val.append(len(add_occupations))  # число доп видов деятельности
    if add_occupations:
        val.append(add_occupations[0].attrib['ВерсОКВЭД'])  # 2014 = okovd2   
        val.append([el.attrib['КодОКВЭД'] for el in add_occupations])  # список кодов
    else:
        # если нет добавочной деятельности => нет и соотв версии
        val.append(None)
        # и список доп.кодов пустой
        val.append([])

    # дальше - редкие признаки
    # сведения о лицензиях - число лицензий
    val.append(len(doc.findall('СвЛиценз')))

    # сведения о производимой продкукции
    # число типов продукции, список их кодов, если есть хотя бы 1 ПрОтнПрод == 1 - бинарный признак: 
    # производит инновац/высокотехнологич продукцию
    products = doc.findall('СвПрод')
    val.append(len(products))
    if products:
        # список кодов
        val.append([el.attrib['КодПрод'] for el in products])
        # высокотехнологичный? 1 - да, 2 - нет, аналогично меняю на 1-0
        val.append(1 if '1' in [el.attrib['ПрОтнПрод'] for el in products] else 0)
    else:
        val.append([])
        val.append(0)

    # сведения о контрактах -  ФЗ "О контрактной системе в сфере закупок товаров, работ, услуг 
    # для обеспечения государственных и муниципальных нужд"
    # число контрактов
    # мб - объеденить с договорами?
    val.append(len(doc.findall('СвКонтр')))

    # сведения о договорах - ФЗ "О закупках товаров, работ, услуг отдельными видами юридических лиц" 
    # число договоров
    val.append(len(doc.findall('СвДог')))

    return tuple(val)

In [ ]:
path = '../raw_data/reestr_ms/data-10032023-structure-10032022'
xpath = "Документ[ОргВклМСП]"
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
table_name = 'reestr_msp'
table_columns = ['inn', 'n_employees', 'is_soc_enterprise', 'newly_establ', 'subj_cat', 
                 'subj_form', 'inclusion_date', 'n_okovd_add', 'okovd_add_vers', 'okovd_add_code', 
                 'n_licenses', 'n_products', 'products_code', 'is_high_tech', 'n_contracts', 
                 'n_agreements']

In [5]:
# функция для итерирования по файлам, сбора документов и вставки в таблицу
# тоже разбить на несколько небольших функций?
def insert_to_pgsql(path, xpath, connect, table_name, table_columns):
    files = [path + '/' + file for file in os.listdir(path)]
    for file in files:
        tree = etree.parse(file)
        root = tree.getroot()
        docs = root.findall(xpath)
        
#         print(len(docs), file)
        # если список не пуст:
        if docs:
            values = []
            for doc in docs:
                val = get_val(doc)
                values.append(val)
            
            # транзакции не равномерны - от 1-2 до 900 значений
            # и стоит ли каждый раз закрывать курсор?
            with connect:
                with connect.cursor() as cur:
                    insert = (
                         sql.SQL(
                          'INSERT INTO {} ({}) VALUES {}')
                          .format(sql.Identifier(table_name), 
                                  sql.SQL(',').join(map(sql.Identifier, table_columns)), 
                                  sql.SQL(',').join(map(sql.Literal, values))
                         )
                        )
                    cur.execute(insert)

    connect.close()

- Wall time: 19min 14s, count 2 332 303

In [ ]:
# %%time
# insert_to_pgsql(path, xpath, connect, table_name, table_columns)

- меняем ключ

In [ ]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)

In [ ]:
# одинаковые инн
with connect:
    with connect.cursor() as cur:
        select = 'SELECT inn, count(inn) FROM reestr_msp rm GROUP BY inn HAVING count(inn) > 1;'
        cur.execute(select)
        print(cur.fetchall())

    [(6371005166, 2), (6319000782, 2)]

In [ ]:
# удаляем все 4 записи
with connect:
    with connect.cursor() as cur:
        delete = 'DELETE FROM reestr_msp WHERE inn in (6371005166, 6319000782);'
        cur.execute(delete)

In [ ]:
# и меняем таблицу - удаляем старый первичный ключ, добавляем новый
# это долго - так как проверяет все inn на условие...
with connect:
    with connect.cursor() as cur:
        alter1 = 'ALTER TABLE reestr_msp DROP COLUMN temp_id;'
        alter2 = 'ALTER TABLE reestr_msp ADD PRIMARY KEY (inn);'
        cur.execute(alter1)
        cur.execute(alter2)

In [ ]:
connect.close()

#### - Сведения о суммах недоимки и задолженности по пеням и штрафам

https://www.nalog.gov.ru/opendata/7707329152-debtam/

- Сведения указываются по состоянию на 31 декабря года, предшествующего году их размещения на сайте ФНС России, при наличии такой недоимки и задолженности по пеням и штрафам по состоянию на 1 октября года размещения указанных сведений на сайте ФНС России

- что-то из этого может быть целью

- здесь только юр лица

Собираем:
    
- Документ:
    - атрибут ДатаСост - Дата, по состоянию на которую, подготовлены данные для публикации
    
    - СведНП 
        - атрибут ИННЮЛ - id, очистка и уникальность
    
    - несколько СведНедоим суммируем по атрибутам: 
      
        - СумШтраф - конкретная сумма, ненадлежащее исполнение
        - СумПени - за дни просрочки
        - СумНедНалог - сама недоимка
        
        Сами СведНедоим разбиты по имени налога ("Страховые и другие взносы на обязательное пенсионное страхование, зачисляемые в Пенсионный фонд Российской Федерации", "НЕНАЛОГОВЫЕ ДОХОДЫ, администрируемые налоговыми органами"), при этом какой-либо код отсутствует, а искать по длинной строке с возможными опечатками стоит только, если нужно что-то конкретное.

In [13]:
file = '../raw_data\debt\VO_OT6\VO_OTKRDAN6_9965_9965_20221220_000cc2e4-2682-40ab-9982-98870d1423d9.xml'
tree = etree.parse(file)
root = tree.getroot()
docs = root.findall("Документ")
doc = docs[0]

In [14]:
print(etree.tostring(doc, encoding = "unicode", pretty_print=True))

<Документ ИдДок="30141970-7ee6-44c9-8bb6-5db64b0576c1" ДатаДок="20.12.2022" ДатаСост="31.12.2021">
  <СведНП НаимОрг="ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot;СУББОТНИК&quot;" ИННЮЛ="7430020629"/>
  <СведНедоим НаимНалог="Страховые взносы на обязательное медицинское страхование работающего населения, зачисляемые в бюджет Федерального фонда обязательного медицинского страхования" СумНедНалог="0.00" СумПени="56.14" СумШтраф="0.00" ОбщСумНедоим="56.14"/>
  <СведНедоим НаимНалог="НЕНАЛОГОВЫЕ ДОХОДЫ, администрируемые налоговыми органами" СумНедНалог="0.00" СумПени="0.00" СумШтраф="500.00" ОбщСумНедоим="500.00"/>
  <СведНедоим НаимНалог="Налог, взимаемый в связи с  применением упрощенной  системы налогообложения" СумНедНалог="47026.42" СумПени="29529.77" СумШтраф="16360.00" ОбщСумНедоим="92916.19"/>
  <СведНедоим НаимНалог="Страховые и другие взносы на обязательное пенсионное страхование, зачисляемые в Пенсионный фонд Российской Федерации" СумНедНалог="0.00" СумПени="242.16" СумШтраф="

In [ ]:
# для обработки сумм
def to_float(val):
    if isinstance(val, str):
        val = val.strip().replace(',', '.')      
        try:
            return float(val)        
        except ValueError:                                           
            return 0 # val  # чтобы не терять другие значения
    return val

In [ ]:
def get_val(doc):
    """
    Принимает один документ
    Возвращает необходимые поля, упакованые в кортеж
    """
    val = []
    # инн = ключ
    val.append(clean(doc.find('СведНП').attrib['ИННЮЛ']))
    # дата, по состоянию на которую есть данные
    val.append(clean(doc.attrib['ДатаСост']))
    # общая сумма по видам налогов: сама недоимка, пени (за дни просрочки), 
    # штраф (конкретная сумма, ненадлежащее исполнение)
    all_debts = doc.findall('СведНедоим')
    # недоимка
    arrears = 0
    # пени
    penalties = 0
    # штраф
    fines = 0
    for debt in all_debts:
        arrears += to_float(debt.attrib['СумНедНалог'])
        penalties += to_float(debt.attrib['СумПени'])
        fines += to_float(debt.attrib['СумШтраф'])
    val.extend([arrears, penalties, fines])
    
    return tuple(val)

In [ ]:
# таблица
debt_sql = 'CREATE TABLE debt (\
inn int8 PRIMARY KEY, \
status_date date, \
arrears money, \
penalties money, \
fines money \
);'

In [ ]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
with connect:
    with connect.cursor() as cur:
        cur.execute(debt_sql)

connect.close()

In [ ]:
path = '../raw_data/debt/VO_OT6'
xpath = "Документ"  # содержит только ИННЮЛ
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
table_name = 'debt'
table_columns = ['inn', 'status_date', 'arrears', 'penalties', 'fines']

In [ ]:
# %%time
# insert_to_pgsql(path, xpath, connect, table_name, table_columns)

- Wall time: 5min 50s, count 1 080 232

#### - реестр субъектов малого и среднего предпринимательства – получателей поддержки

https://www.nalog.gov.ru/opendata/7707329152-rsmppp/

- опять же, много видов поддержки, возьмём только финансовую

- Документ (у которого есть элемент СвЮЛ)
    - СвЮЛ - атрибут ИННЮЛ
    - и дальше несколько СвПредПод
        
        - атрибут ИнфНаруш - 1 или 2 - Информация о наличии нарушения порядка и условий оказания поддержки
        - атрибут ДатаПрин - дата принятия решения о предоставлении поддержки

        - элемент ФормПод - форма поддержки

        отбираем тех, у кого атрибут КодФорм="0100" - это финансовая поддержка
        - на этом же уровне - элемент ВидПод - 
            - субсидии и гранты, КодВид="0103" ед 1, видимо деньги
            - финансирование на возвратной основе КодВид="0104" ед поддержки 4
            - гарантии и поручительство ="0102"
        - РазмПод имеет значения в зависимости от атрибута ЕдПод - Единица измерения поддержки, значение 1 (деньги), 2, 3 (видимо, число консультаций), 4 (не знаю, что, процент?), 5 (мероприятия?)

- может быть несколько поддержек для 1 организации одного вида или разных видов
- так как не знаю, что ищу, можно сделать строка = конкретный акт поддержки (а не организация)
- тем более, что фактор времени (дата принятия решения о поддержке) - пока не знаю, какой период нужен

In [9]:
file = "../raw_data/support_ms/data/VO_SVMSP_0000_9965_20230315_00a60c8b-cb46-42f4-bb16-beee983cf34d.xml"
tree = etree.parse(file)
root = tree.getroot()
docs = root.findall("Документ[СвЮЛ]")
doc = docs[1]

In [10]:
print(etree.tostring(doc, encoding = "unicode", pretty_print=True))

<Документ ИдДок="264abcc9-c501-4845-89df-3d77cdbb15a6" ДатаСост="15.03.2023">
  <СвЮЛ ИННЮЛ="3810047203" НаимОрг="ООО &quot;БАРГУЗИН&quot;"/>
  <СвПредПод ВидПП="1" НаимОрг="ФЕДЕРАЛЬНАЯ НАЛОГОВАЯ СЛУЖБА" ИННЮЛ="7707329152" КатСуб="1" СрокПод="19.06.2020" ДатаПрин="19.06.2020" ИнфНаруш="2">
    <ФормПод КодФорм="0100" НаимФорм="Финансовая поддержка"/>
    <ВидПод КодВид="0103" НаимВид="Предоставление субсидий и грантов"/>
    <РазмПод РазмПод="48520.00" ЕдПод="1"/>
  </СвПредПод>
  <СвПредПод ВидПП="1" НаимОрг="ФЕДЕРАЛЬНАЯ НАЛОГОВАЯ СЛУЖБА" ИННЮЛ="7707329152" КатСуб="1" СрокПод="19.05.2020" ДатаПрин="19.05.2020" ИнфНаруш="2">
    <ФормПод КодФорм="0100" НаимФорм="Финансовая поддержка"/>
    <ВидПод КодВид="0103" НаимВид="Предоставление субсидий и грантов"/>
    <РазмПод РазмПод="48520.00" ЕдПод="1"/>
  </СвПредПод>
  <СвПредПод ВидПП="1" НаимОрг="ФЕДЕРАЛЬНАЯ НАЛОГОВАЯ СЛУЖБА" ИННЮЛ="7707329152" КатСуб="1" СрокПод="25.11.2021" ДатаПрин="25.11.2021" ИнфНаруш="2">
    <ФормПод КодФорм="010

In [15]:
def get_values(doc):
    """
    Принимает документ
    Возвращает несколько строк для вставки (список кортежей) или ни одной строки
    """   
    values = []
    # общие данные для строк одного документа
    inn = clean(doc.find('СвЮЛ').attrib['ИННЮЛ'])
    # вся финансовая поддержка
    supports = doc.findall('СвПредПод')

    for support in supports:
        # если финансовая поддержка - забираем
        if support.find('ФормПод').attrib['КодФорм'] == "0100":
            val = []
            val.append(inn)
            # дата принятия решения о предоставлении поддержки
            val.append(support.attrib['ДатаПрин'])
            # есть ли нарушение? 1 - да, 0 - нет
            val.append(1 if support.attrib['ИнфНаруш'] == "1" else 0)
            # код формы: 0102 - гарантии и поручительство, 0103 - субсидии и гранты, 
            # 0104 - возвратная основа
            val.append(clean(support.find('ВидПод').attrib['КодВид']))
            # размер
            val.append(to_float(support.find('РазмПод').attrib['РазмПод']))
            # единица: 1 (деньги), 2, 3 (число консультаций?), 4 (процент?), 5 (мероприятия?)
            # 1 для 0102 и 0103, 4 для 0104
            val.append(clean(support.find('РазмПод').attrib['ЕдПод']))

            values.append(tuple(val))
    
    return values

In [16]:
# и своя ф-я для вставки, ещё более громоздкая!
def inserts_to_pgsql(path, xpath, connect, table_name, table_columns):
    files = [path + '/' + file for file in os.listdir(path)]
    for file in files:
        tree = etree.parse(file)
        root = tree.getroot()
        docs = root.findall(xpath)
        
#         print(len(docs), file)
        # если список не пуст:
        if docs:
            values = []
            for doc in docs:
                val_list = get_values(doc)
                # если не пуст
                if val_list:
                    values.extend(val_list)

            if values:
                with connect:
                    with connect.cursor() as cur:
                        insert = (
                             sql.SQL(
                              'INSERT INTO {} ({}) VALUES {}')
                              .format(sql.Identifier(table_name), 
                                      sql.SQL(',').join(map(sql.Identifier, table_columns)), 
                                      sql.SQL(',').join(map(sql.Literal, values))
                             )
                            )
                        cur.execute(insert)

    connect.close()

In [17]:
support_sql = 'CREATE TABLE support_ms (\
id bigserial PRIMARY KEY, \
inn int8, \
support_date date, \
is_breach int2, \
form_code char (4), \
support_volume money, \
support_unit int2 \
);'

In [ ]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
with connect:
    with connect.cursor() as cur:
        cur.execute(support_sql)

connect.close()

In [ ]:
path = '../raw_data/support_ms/data'
xpath = "Документ[СвЮЛ]"
connect = psycopg2.connect(dbname="fns_data", user="user", password='new_password', host="localhost", port=5432)
table_name = 'support_ms'
table_columns = ['inn', 'support_date', 'is_breach', 'form_code', 'support_volume', 'support_unit']

In [ ]:
# %%time
# inserts_to_pgsql(path, xpath, connect, table_name, table_columns)

- Wall time: 10min 23s
- Count 1 170 650, count(DISTINCT inn) - 381 279
- уникальные даты поддержки - с 2019 по 2023

- на данный момент - 3 таблицы, будут связаны по inn

- в MongoDB около 300 тыс строк, к моменту, как начну соединять данные, думаю, будет около 500 тыс
- необработаны остались 2 xml архива и немного csv (много проще структура)